In [0]:
!pip install kaggle

In [0]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c dogs-vs-cats

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 11, in <module>
    load_entry_point('kaggle==1.5.3', 'console_scripts', 'kaggle')()
  File "/usr/local/lib/python2.7/dist-packages/pkg_resources/__init__.py", line 489, in load_entry_point
    return get_distribution(dist).load_entry_point(group, name)
  File "/usr/local/lib/python2.7/dist-packages/pkg_resources/__init__.py", line 2793, in load_entry_point
    return ep.load()
  File "/usr/local/lib/python2.7/dist-packages/pkg_resources/__init__.py", line 2411, in load
    return self.resolve()
  File "/usr/local/lib/python2.7/dist-packages/pkg_resources/__init__.py", line 2417, in resolve
    module = __import__(self.module_name, fromlist=['__name__'], level=0)
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/us

In [0]:
!ls

sample_data


In [0]:
!unzip train.zip

unzip:  cannot find or open train.zip, train.zip.zip or train.zip.ZIP.


In [0]:
!unzip test1.zip

unzip:  cannot find or open test1.zip, test1.zip.zip or test1.zip.ZIP.


In [0]:
!ls

sample_data


In [0]:
import cv2
import os
import math
from google.colab.patches import cv2_imshow
import numpy as np

trainPath = "/content/train/"
testPath = "/content/test1/"

class Knn(object):
  def __init__(self):
    pass
  
  #A function to load and prepare the data to be predicted
  def train(self, path):
    
    #listdir() returns a list containing the names of the entries in the directory given by the path.
    imagesList = os.listdir(trainPath)
    
    rawImgSum = []
    features = []
    labels = []

    for itr in imagesList:

      image = cv2.imread(trainPath + itr) # AS: Why not path?
      
      image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      rawImgSum.append(cv2.resize(image_gray, (32, 32)).flatten())
      
      image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
      hist = cv2.calcHist([image_hsv], [0, 1, 2], None, (8, 8, 8), [0, 180, 0, 256, 0, 256])
      features.append(cv2.resize(hist, (32,32)).flatten())

      labels.append(itr.split('.')[0])

    self.trImSum = rawImgSum
    self.trFeSum = features
    self.trLabel = labels
  
  
  #A function to predict the label of the test image using raw intensities of the pixels
  def predRaw(self, path, k):
    
    distances = []
    
    im = cv2.imread(path)
    image_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    testImSum = cv2.resize(image_gray, (32,32)).flatten()

    for i in range(len(self.trImSum)):
      distances.append(np.sqrt(np.sum(np.square(abs(np.array(self.trImSum[i]) - np.array(testImSum)))))) # AS: you don't need to get the sqrt
      
    minIndex = []
    
    for i in range(k):
      print("New minimum distance = " + str(min(distances)) + " , " + "Index of the new minimum distance = " + str(distances.index(min(distances))))
      minIndex.append(distances.index(min(distances)))
      distances[minIndex[i]] = 1000000000
    
    cat = 0
    dog = 0
    
    for i in range(len(minIndex)):
            
      if self.trLabel[minIndex[i]] == "cat":
        cat +=1
        print("Label :" + self.trLabel[minIndex[i]] + " = " + str(cat))
      elif self.trLabel[minIndex[i]] == "dog":
        dog +=1
        print("Label :" + self.trLabel[minIndex[i]] + " = " + str(dog))
      
    if cat > dog:
      print("This is a cat", cat)
    elif dog > cat:
      print("This is a dog", dog)
    else:
      print("Not Sure!")
      
    cv2_imshow(im)
    
    
  #A function to predict the label of the test image using color histogram feature  
  def predFeat(self, path, k):
    
    distances = []
    
    im = cv2.imread(path)
    testImSum = sum(cv2.resize(im, (32,32)).flatten()[:])

    for i in range(len(self.trFeSum)):
      distances.append(np.sqrt(np.sum(np.square(abs(np.array(self.trFeSum[i]) - np.array(testImSum))))))
      
    minIndex = []
    
    for i in range(k):
      
      print("New minimum distance = " + str(min(distances)) + " , " + "Index of the new minimum distance = " + str(distances.index(min(distances))))
      minIndex.append(distances.index(min(distances)))
      distances[minIndex[i]] = 1000000000
    
    cat = 0
    dog = 0
    
    for i in range(len(minIndex)):
            
      if self.trLabel[minIndex[i]] == "cat":
        cat +=1
        print("Label :" + self.trLabel[minIndex[i]] + " = " + str(cat))
      elif self.trLabel[minIndex[i]] == "dog":
        dog +=1
        print("Label :" + self.trLabel[minIndex[i]] + " = " + str(dog))

      
    if cat > dog:
      print("This is a cat", cat)
    elif dog > cat:
      print("This is a dog", dog)
    else:
      print("Not Sure!")
      
    cv2_imshow(im)

In [0]:
#Training takes a while
classifer = Knn()
classifer.train(trainPath)

In [0]:
testImages = os.listdir(testPath)

In [0]:
#The Accuracy of the classifier increases when k increases, try 1, 10, 100, 1000 for k
#testImages is a list contatining the names of all test images from 0 to 12499
classifer.predRaw(testPath + testImages[0], 7)

In [0]:
#Using color Histogram
#The Accuracy of the classifier increases when k increases, try 1, 10, 100, 1000 for k
#testImages is a list contatining the names of all test images from 0 to 12499
classifer.predFeat(testPath + testImages[0], 7)